In [ ]:
source("predictors_SC1.R")
source("myutils.R")
library(caret)
options(warn=-1)

In [2]:
nfolds <- 5
npatients <- dim(read.table(paste("../data/features/prepared_data_dna.csv",sep=""),sep="\t",header=TRUE,row.names=1))[1]
patients_ind <- 1:npatients
patients_names <- rownames(read.table(paste("../data/features/prepared_data_dna.csv",sep=""),sep="\t",header=TRUE,row.names=1))
chemicals <- colnames(read.table("../data/responses/prepared_data_auc.csv"))

flds <- createFolds(1:npatients, nfolds,list=TRUE)

for (y in 1:length(flds)){

    index_Test <- patients_ind[flds[[y]]]
    rest <- unlist(flds[-y])
    names(rest) <- NULL
    index_Train <- patients_ind[rest]
                   
    features_model<- list(c("RNA"))
    predictors <- c(predictorElasticNet,predictorRF,predictorSVR)
    str_predictors <- c("predictorElasticNet","predictorRF","predictorSVR")

    # Parameters for ElasticNet:
    l_alpha <-seq(0.05,0.95,0.1)
    
    # Parameters for RF:
    l_ntree <- seq(150,1000,50)
    l_nodesize <- seq(5,25,5)

    # Parameters for XGBoost:
    l_max.depth <- seq(1,10,1)
    l_eta <- seq(0,1,0.1)
    l_nthread <- 2
    l_nrounds <- seq(5,145,10)

    # Parameters for kNN:
    l_k = 2:25

    # Parameters for SVR:
    l_kernel <- c("radial","polynomial")

    drug_responses <- "all" # at least two drugs must be specified. Otherwise nchemicals in the evaluate function cannot be computed
    launch_prediction(y, features_model, drug_responses=drug_responses,
                      predictors, str_predictors, paste(index_Train,collapse="_"), paste(index_Test,collapse="_"),
                      l_k=l_k, l_ntree=l_ntree, l_nodesize=l_nodesize, l_alpha=l_alpha,
                      l_max.depth=l_max.depth, l_eta=l_eta,l_nthread=l_nthread,
                      l_nrounds=l_nrounds,l_kernel=l_kernel) 
}

[1] "bsub -R 'rusage[mem=4]' -W 05:59 -n 4 -J 'model_1' -o '../prediction/myresults_SC1/RNA/times_1/stdoutput/predictorElasticNet_0.05_drugs_all.txt' 'Rscript myevaluate_SC1_intermediate_parallel.R --predictor predictorElasticNet --feature_model RNA --index_train 13_14_15_17_21_28_30_32_35_41_51_55_61_63_71_75_81_89_94_99_103_105_115_116_126_139_145_147_150_152_153_159_160_170_182_183_184_185_190_194_208_211_212_213_5_6_10_18_26_27_29_31_43_44_47_58_59_62_64_68_74_80_86_91_98_106_108_113_114_118_119_121_122_124_130_141_163_166_168_172_175_186_193_199_209_210_1_12_16_20_22_23_36_37_46_53_54_57_66_67_69_70_72_77_82_83_88_97_109_112_117_132_134_137_138_140_144_146_155_165_171_173_174_176_177_179_189_195_205_206_2_4_7_8_11_19_39_45_49_50_56_60_65_79_85_92_95_96_100_101_110_111_125_127_131_133_135_143_148_154_158_162_164_169_178_180_187_200_201_202_203_204 --index_test 3_9_24_25_33_34_38_40_42_48_52_73_76_78_84_87_90_93_102_104_107_120_123_128_129_136_142_149_151_156_157_161_167_181_188_191

In [ ]:
choose <- c("RNA")

for (y in 1:length(flds)){

    index_Test <- patients_ind[flds[[y]]]
    best_results <- list()
    best_results_rmsd <- list()

    for (drug in chemicals){
        
        for (algo in str_predictors){
    
            res <- best_model_per_drug(drug,choose = choose, algo=algo)
            best_results[[drug]] <- res[["spearman"]]
            best_results_rmsd[[drug]] <- res[["rmsd"]]
        
        }
        
        best_results_df <- do.call(rbind,best_results)
        write.table(best_results_df,"./../prediction/myresults_SC1/",paste(choose,collpase="_"),"/times_",y,"/",algo,"_best_models.csv",row.names=T,col.names=T,sep="\t")

        best_results_df_rmsd <- do.call(rbind,best_results_rmsd)
        write.table(best_results_df,"./../prediction/myresults_SC1/",paste(choose,collpase="_"),"/times_",y,"/",algo,"_best_models_rmsd.csv",row.names=T,col.names=T,sep="\t")
    }


}

In [6]:
chemicals <- colnames(read.table("../data/responses/prepared_data_auc.csv"))
best_results <- list()

for (drug in chemicals){
    
    best_results[[drug]] <- best_model_per_drug(drug)
}

best_results_df <- do.call(rbind,best_results)
write.table(best_results_df,"./../docker/docker/best_models.csv",row.names=T,col.names=T,sep="\t")

In [9]:
data_response <- read.table("./../data/responses/prepared_data_auc.csv")
data_matrix <- read.table("./../docker/docker/best_models.csv")
save_trained_best_models(data_response,data_matrix)

[1] 213  71
[1] 213  11
[1] 213  71
[1] 213  71
[1] 213  11
[1] 213   2
[1] 213  71
[1] 213  71
[1]	train-rmse:32.199516 
[2]	train-rmse:15.263824 
[3]	train-rmse:8.654536 
[4]	train-rmse:3.428747 
[5]	train-rmse:2.326570 
[1] 213  52
[1] 213   2
[1] 213  11
[1] 213  71
[1] 213  11
[1] 213  71
[1] 213  71
[1] 213   2
[1] 213  71
[1] 213  52
[1] 213   2
[1] 213  11
[1] 213  52
[1] 213  11
[1]	train-rmse:41.254448 
[2]	train-rmse:35.200516 
[3]	train-rmse:30.488880 
[4]	train-rmse:28.366991 
[5]	train-rmse:26.508612 
[1] 213   2
[1] 213  71
[1]	train-rmse:53.335052 
[2]	train-rmse:51.218910 
[3]	train-rmse:49.693253 
[4]	train-rmse:47.942966 
[5]	train-rmse:46.290733 
[6]	train-rmse:44.670456 
[7]	train-rmse:42.874939 
[8]	train-rmse:41.412045 
[9]	train-rmse:40.229206 
[10]	train-rmse:38.853886 
[11]	train-rmse:37.807255 
[12]	train-rmse:36.729950 
[13]	train-rmse:35.587318 
[14]	train-rmse:34.752853 
[15]	train-rmse:34.030365 
[1] 213  71
[1] 213   2
[1] 213  11
[1] 213  71
[1] 213   2

In [27]:
library(datasets)
library(caret)
data(iris)
iris

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa
4.6,3.4,1.4,0.3,setosa
5.0,3.4,1.5,0.2,setosa
4.4,2.9,1.4,0.2,setosa


In [30]:
flds <- createFolds(1:dim(iris)[1], 10,list=TRUE)
flds


$Fold01
 [1]  30  32  35  46  53  60  70  76  78 106 108 116 127 129 144

$Fold02
 [1]   2  12  33  34  43  63  64  73  83  84 105 112 121 126 135

$Fold03
 [1]   4   6   8   9  39  55  56  74  79  81  88  99 113 117 123 137

$Fold04
 [1]  19  23  28  31  42  45  65  72  92 100 111 114 125 140 149

$Fold05
 [1]   5  22  25  26  48  54  71  82  89  93  97 115 128 132 143

$Fold06
 [1]  14  21  24  51  52  61  75  80  85  87  95 122 131 138 146

$Fold07
 [1]   3  10  15  18  41  44  50  66  86  90  96 120 130 133 139

$Fold08
 [1]  13  17  27  38  49  58  59  62  77 101 109 118 136 145 150

$Fold09
 [1]  16  20  36  37  40  68  69  94  98 102 103 119 134 141 147

$Fold10
 [1]   1   7  11  29  47  57  67  91 104 107 110 124 142 148

In [20]:
dim(iris)[1]

createDataPartition {caret},R Documentation
y,"a vector of outcomes. For createTimeSlices, these should be in chronological order."
times,the number of partitions to create
p,the percentage of data that goes to training
list,logical - should the results be in a list (TRUE) or a matrix with the number of rows equal to floor(p * length(y)) and times columns.
groups,"for numeric y, the number of breaks in the quantiles (see below)"
k,an integer for the number of folds.
returnTrain,"a logical. When true, the values returned are the sample positions corresponding to the data used during training. This argument only works in conjunction with list = TRUE"
initialWindow,The initial number of consecutive values in each training set sample
horizon,the number of consecutive values in test set sample
fixedWindow,"logical, if FALSE, all training samples start at 1"
